<a href="https://colab.research.google.com/github/96jonesa/StyleGan2-Colab-Demo/blob/master/output_small_set_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# All of the models were trained using a notebook I wrote to allow for easy training
# using the free resources provided by Colab and Google Drive, linked here:
#
# https://colab.research.google.com/drive/1QWMCwSIdDb3GQp7lHqWc0RAcnI2FANBw?usp=sharing
#
# Using the PyTorch implementation of StyleGAN2 available at:
#
# https://github.com/lucidrains/stylegan2-pytorch

In [ ]:
# How to use:
#
# Login to Google (Drive).
#
# Image displays are full size 8x8 grids of 128x128 images (32x32 in case of cifar10).
# You will need to scroll through the cell output to see each of the (typically 4)
# grids displayed in each output. Simply place you cursor somewhere inside the output,
# then scroll.
#
# You can either run all the cells then scroll through with 'Runtime > Run all'
#   (this will cause a lot of images to print and can be tedious to scroll through)
#
# Or you can step through cell-by-cell by running each cell individually with CMD+ENTER

In [ ]:
# Descriptions of datasets (citations at end of notebook):
#
# metfaces: "image dataset of human faces extracted from works of art" -NVlabs
# celeba: "large-scale face attributes dataset with more than 200K celebrity images" -Lie et al.
# afhq_dog: "5,000 high-quality images [of dogs] at 512×512 resolution" -Choi et al.
# cifar10_horse: 5000 32x32 color images of horses -Krizhevsky

In [ ]:
# Utilities for downloading publicly shared Google Drive files (from my Google Drive).

import requests

def download_file_from_google_drive(id, destination):
    URL = 'https://docs.google.com/uc?export=download'

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, 'wb') as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [ ]:
import zipfile
from IPython.display import Image, display

file_id = '1uEuLbFWRdAbaB2p3pvQOUUMj7_FJF3bN'
destination = 'demo_samples.zip'
download_file_from_google_drive(file_id, destination)
zip_ref = zipfile.ZipFile('demo_samples.zip', 'r')
zip_ref.extractall('demo_samples')
zip_ref.close()

In [ ]:
# First, we will look at sample images generated by the pretrained models using no differentiable augmentation and no attention.
# We expect the results to lack diversity (due to the lack of differentiable augmentation) and be of relatively poor quality
# (due to the lack of attention). In order, these 8x8 image displays are from training on the following datasets:
#
# 1. cifar10_horse  (32x32 resolution, small dataset (5000 images), 97000 iterations @ 4.52 it/s on P100 GPU)
# 2. afhq_dog       (128x128 resolution, small dataset (5000 images), 38000 iterations @ 1.78 it/s on P100 GPU)
# 3. metfaces       (128x128 resolution, small dataset (1336 images), 27000 iterations @ 1.27 it/s on P100 GPU)
# 4. celeba         (128x128 resolution, large dataset (202599 images), 36000 iterations @ 1.64 it/s on P100 GPU)

print('cifar10_horse, no differentiable augmentation, no attention, standard')
display(Image('/content/demo_samples/StyleGan2_small_set_demo_samples/cifar10_horse_aug_00_attn_none_97.jpg'))
print()
print()
print('afhq_dog, no differentiable augmentation, no attention, standard')
display(Image('/content/demo_samples/StyleGan2_small_set_demo_samples/afhq_dog_aug_00_attn_none_38.jpg'))
print()
print()
print('metfaces, no differentiable augmentation, no attention, standard')
display(Image('/content/demo_samples/StyleGan2_small_set_demo_samples/metfaces_aug_00_attn_none_27.jpg'))
print()
print()
print('celeba, no differentiable augmentation, no attention, standard')
display(Image('/content/demo_samples/StyleGan2_small_set_demo_samples/celeba_aug_00_attn_none_36.jpg'))

In [ ]:
# Notice the poor quality? The model also generates samples from a version of the model which applies multiplicative
# Gaussian noise to each convolutional layer in the discriminator, with the magnitude of the noise is a function of
# the exponential moving average of the discriminator output. This improves stability and quality significantly.
# The result is shown below for the same models as above:

print('cifar10_horse, no differentiable augmentation, no attention, EMA noise')
display(Image('/content/demo_samples/StyleGan2_small_set_demo_samples/cifar10_horse_aug_00_attn_none_97-ema.jpg'))
print()
print()
print('afhq_dog, no differentiable augmentation, no attention, EMA noise')
display(Image('/content/demo_samples/StyleGan2_small_set_demo_samples/afhq_dog_aug_00_attn_none_38-ema.jpg'))
print()
print()
print('metfaces, no differentiable augmentation, no attention, EMA noise')
display(Image('/content/demo_samples/StyleGan2_small_set_demo_samples/metfaces_aug_00_attn_none_27-ema.jpg'))
print()
print()
print('celeba, no differentiable augmentation, no attention, EMA noise')
display(Image('/content/demo_samples/StyleGan2_small_set_demo_samples/celeba_aug_00_attn_none_36-ema.jpg'))

In [ ]:
# The model also generates samples using this EMA noise as well as mixed regularities (using two latent space noise vectors
# as input to the generator, and switching from one to the other at a fixed point in the synthesis, essentially mixing
# the styles which would be generated by the two), which causes a decorrelation of neighboring styles and thus allows
# for more fine-tuned diversity (at the cost of a bit of image quality from the amount of training):

print('cifar10_horse, no differentiable augmentation, no attention, mixed regularities')
display(Image('/content/demo_samples/StyleGan2_small_set_demo_samples/cifar10_horse_aug_00_attn_none_97-mr.jpg'))
print()
print()
print('afhq_dog, no differentiable augmentation, no attention, mixed regularities')
display(Image('/content/demo_samples/StyleGan2_small_set_demo_samples/afhq_dog_aug_00_attn_none_38-mr.jpg'))
print()
print()
print('metfaces, no differentiable augmentation, no attention, mixed regularities')
display(Image('/content/demo_samples/StyleGan2_small_set_demo_samples/metfaces_aug_00_attn_none_27-mr.jpg'))
print()
print()
print('celeba, no differentiable augmentation, no attention, mixed regularities')
display(Image('/content/demo_samples/StyleGan2_small_set_demo_samples/celeba_aug_00_attn_none_36-mr.jpg'))

In [ ]:
# By now you have likely noticed the extremely low quality of the afhq_dog model. This model has so little data to work with
# that it degenerates after only a few hours of training. Here are the EMA noise and mixed regularities outputs from an
# earlier training checkpoint of the same model:

print('cifar10_horse early stop, no differentiable augmentation, no attention, EMA noise')
display(Image('/content/demo_samples/StyleGan2_small_set_demo_samples/afhq_dog_aug_00_attn_none_22-ema.jpg'))
print()
print()
print('cifar10_horse early stop, no differentiable augmentation, no attention, mixed regularities')
display(Image('/content/demo_samples/StyleGan2_small_set_demo_samples/afhq_dog_aug_00_attn_none_22-mr.jpg'))

In [ ]:
# By now you have likely noticed the extreme lack of diversity in the afhq_dog and metfaces models, as well as a more
# subtle lack of diversity in the cifar10_horses and celeba models. This is due to the small amount of data available in
# the former two datasets, and a reasonable yet still lacking amount of data in the latter two datasets.
# A recent innovation in data augmentation which uses differentiable augmentations of the data has led to models obtaining
# high quality results with up to 70x less data. The following show the results of training the above models while applying
# this augmentation with probability 0.2 to at each iteration (the EMA noise results are shown, without mixed regularities).
# In order, these were trained on:
#
# 1. cifar10_horse  (32x32 resolution, small dataset (5000 images), 97000 iterations @ 4.51 it/s on P100 GPU)
# 2. afhq_dog       (128x128 resolution, small dataset (5000 images), 38000 iterations @ 1.56 it/s on P100 GPU)
# 3. metfaces       (128x128 resolution, small dataset (1336 images), 27000 iterations @ 1.05 it/s on P100 GPU)
# 4. celeba         (128x128 resolution, large dataset (202599 images), 36000 iterations @ 1.64 it/s on P100 GPU)

print('cifar10_horse, 0.2 differentiable augmentation, no attention, EMA noise')
display(Image('/content/demo_samples/StyleGan2_small_set_demo_samples/cifar10_horse_aug_02_attn_none_97-ema.jpg'))
print()
print()
print('afhq_dog, 0.2 differentiable augmentation, no attention, EMA noise')
display(Image('/content/demo_samples/StyleGan2_small_set_demo_samples/afhq_dog_aug_02_attn_none_38-ema.jpg'))
print()
print()
print('metfaces, 0.2 differentiable augmentation, no attention, EMA noise')
display(Image('/content/demo_samples/StyleGan2_small_set_demo_samples/metfaces_aug_02_attn_none_27-ema.jpg'))
print()
print()
print('celeba, 0.2 differentiable augmentation, no attention, EMA noise')
display(Image('/content/demo_samples/StyleGan2_small_set_demo_samples/celeba_aug_02_attn_none_36-ema.jpg'))

In [ ]:
# The same augmented data models produced the following sample images using mixed regularities:

print('cifar10_horse, 0.2 differentiable augmentation, no attention, mixed regularities')
display(Image('/content/demo_samples/StyleGan2_small_set_demo_samples/cifar10_horse_aug_02_attn_none_97-mr.jpg'))
print()
print()
print('afhq_dog, 0.2 differentiable augmentation, no attention, mixed regularities')
display(Image('/content/demo_samples/StyleGan2_small_set_demo_samples/afhq_dog_aug_02_attn_none_38-mr.jpg'))
print()
print()
print('metfaces, 0.2 differentiable augmentation, no attention, mixed regularities')
display(Image('/content/demo_samples/StyleGan2_small_set_demo_samples/metfaces_aug_02_attn_none_27-mr.jpg'))
print()
print()
print('celeba, 0.2 differentiable augmentation, no attention, mixed regularities')
display(Image('/content/demo_samples/StyleGan2_small_set_demo_samples/celeba_aug_02_attn_none_36-mr.jpg'))

In [ ]:
# COMING SOON:
#
# Results from training using attention on every layer, with and without augmentation.
#
# Results from training with various other differentiable augmentation probabilities (0.1 and 0.3).
#
# Results from training for more iterations under these configurations.

```
CITATIONS:

@inproceedings{choi2020starganv2,
  title={StarGAN v2: Diverse Image Synthesis for Multiple Domains},
  author={Yunjey Choi and Youngjung Uh and Jaejun Yoo and Jung-Woo Ha},
  booktitle={Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition},
  year={2020}
}

@inproceedings{liu2015faceattributes,
 title = {Deep Learning Face Attributes in the Wild},
 author = {Liu, Ziwei and Luo, Ping and Wang, Xiaogang and Tang, Xiaoou},
 booktitle = {Proceedings of International Conference on Computer Vision (ICCV)},
 month = {December},
 year = {2015} 
}

@article{Karras2019stylegan2,
  title   = {Analyzing and Improving the Image Quality of {StyleGAN}},
  author  = {Tero Karras and Samuli Laine and Miika Aittala and Janne Hellsten and Jaakko Lehtinen and Timo Aila},
  journal = {CoRR},
  volume  = {abs/1912.04958},
  year    = {2019},
}

@misc{zhao2020feature,
    title   = {Feature Quantization Improves GAN Training},
    author  = {Yang Zhao and Chunyuan Li and Ping Yu and Jianfeng Gao and Changyou Chen},
    year    = {2020}
}

@misc{chen2020simple,
    title   = {A Simple Framework for Contrastive Learning of Visual Representations},
    author  = {Ting Chen and Simon Kornblith and Mohammad Norouzi and Geoffrey Hinton},
    year    = {2020}
}

@article{,
  title     = {Oxford 102 Flowers},
  author    = {Nilsback, M-E. and Zisserman, A., 2008},
  abstract  = {A 102 category dataset consisting of 102 flower categories, commonly occuring in the United Kingdom. Each class consists of 40 to 258 images. The images have large scale, pose and light variations.}
}

@article{afifi201911k,
  title   = {11K Hands: gender recognition and biometric identification using a large dataset of hand images},
  author  = {Afifi, Mahmoud},
  journal = {Multimedia Tools and Applications}
}

@misc{zhang2018selfattention,
    title   = {Self-Attention Generative Adversarial Networks},
    author  = {Han Zhang and Ian Goodfellow and Dimitris Metaxas and Augustus Odena},
    year    = {2018},
    eprint  = {1805.08318},
    archivePrefix = {arXiv}
}

@article{shen2019efficient,
  author    = {Zhuoran Shen and
               Mingyuan Zhang and
               Haiyu Zhao and
               Shuai Yi and
               Hongsheng Li},
  title     = {Efficient Attention: Attention with Linear Complexities},
  journal   = {CoRR},  
  year      = {2018},
  url       = {http://arxiv.org/abs/1812.01243},
}

@misc{zhao2020image,
    title  = {Image Augmentations for GAN Training},
    author = {Zhengli Zhao and Zizhao Zhang and Ting Chen and Sameer Singh and Han Zhang},
    year   = {2020},
    eprint = {2006.02595},
    archivePrefix = {arXiv}
}

@misc{karras2020training,
    title   = {Training Generative Adversarial Networks with Limited Data},
    author  = {Tero Karras and Miika Aittala and Janne Hellsten and Samuli Laine and Jaakko Lehtinen and Timo Aila},
    year    = {2020},
    eprint  = {2006.06676},
    archivePrefix = {arXiv},
    primaryClass = {cs.CV}
}

@article{article,
author = {Krizhevsky, Alex},
year = {2012},
month = {05},
pages = {},
title = {Learning Multiple Layers of Features from Tiny Images},
journal = {University of Toronto}
}

@misc{karras2020training,
    title={Training Generative Adversarial Networks with Limited Data},
    author={Tero Karras and Miika Aittala and Janne Hellsten and Samuli Laine and Jaakko Lehtinen and Timo Aila},
    year={2020},
    eprint={2006.06676},
    archivePrefix={arXiv},
    primaryClass={cs.CV}
}
```


